In [54]:
import logging
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

tf.get_logger().setLevel(logging.ERROR)

In [55]:
data = pd.read_csv('radar.csv')


def _df(data):
    df = pd.DataFrame(data)
    for c in range(df.shape[1]):
        mapping = {df.columns[c]: c}
        df = df.rename(columns=mapping)
    return df


data = (data.drop(columns=["Emitter"])).values
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [56]:
import os
import logging
import numpy as np
import pandas as pd
import tensorflow as tf

tf.get_logger().setLevel(logging.ERROR)


class Gan():

    def __init__(self, data):


        self.data = data
        self.n_epochs = 200

    # Genereta random noise in a latent space
    def _noise(self):
        noise = np.random.normal(0, 1, self.data.shape)
        return noise

    def _generator(self):
        model = tf.keras.Sequential(name="Generator_model")
        model.add(tf.keras.layers.Dense(self.data.shape[1], activation='relu',
                                        kernel_initializer='he_uniform',
                                        input_dim=self.data.shape[1]))
        model.add(tf.keras.layers.Dense(30, activation='relu'))
        model.add(tf.keras.layers.Dense(self.data.shape[1], activation='linear'))
        return model

    def _discriminator(self):
        model = tf.keras.Sequential(name="Discriminator_model")
        model.add(tf.keras.layers.Dense(self.data.shape[1], activation='relu',
                                        kernel_initializer='he_uniform',
                                        input_dim=self.data.shape[1]))
        model.add(tf.keras.layers.Dense(50, activation='relu'))
        # sigmoid => real or fake
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        return model

    # define the combined generator and discriminator model,
    # for updating the generator
    def _GAN(self, generator, discriminator):
        discriminator.trainable = False
        generator.trainable = True
        model = tf.keras.Sequential(name="GAN")
        model.add(generator)
        model.add(discriminator)
        model.compile(loss='binary_crossentropy', optimizer='adam')
        return model

    # train the generator and discriminator
    def train(self, generator, discriminator, gan):
        for epoch in range(self.n_epochs):

            # Train the discriminator
            generated_data = generator.predict(self._noise())
            labels = np.concatenate([np.ones(self.data.shape[0]), np.zeros(self.data.shape[0])])
            X = np.concatenate([self.data, generated_data])
            discriminator.trainable = True
            d_loss , _ = discriminator.train_on_batch(X, labels)

            # Train the generator
            noise = self._noise()
            g_loss = gan.train_on_batch(noise, np.ones(self.data.shape[0]))


            print('>%d, d1=%.3f, d2=%.3f' %(epoch+1, d_loss, g_loss))

        return generator

In [57]:
model = Gan(data=data)
generator = model._generator()
descriminator = model._discriminator()
gan_model = model._GAN(generator=generator, discriminator=descriminator)
trained_model = model.train(
    generator=generator, discriminator=descriminator, gan=gan_model)

6/6 [==============================] - 0s 2ms/step
>1, d1=0.824, d2=0.659
6/6 [==============================] - 0s 3ms/step
>2, d1=0.813, d2=0.661
6/6 [==============================] - 0s 2ms/step
>3, d1=0.801, d2=0.655
6/6 [==============================] - 0s 2ms/step
>4, d1=0.790, d2=0.657
6/6 [==============================] - 0s 2ms/step
>5, d1=0.779, d2=0.658
6/6 [==============================] - 0s 2ms/step
>6, d1=0.771, d2=0.652
6/6 [==============================] - 0s 2ms/step
>7, d1=0.763, d2=0.660
6/6 [==============================] - 0s 2ms/step
>8, d1=0.751, d2=0.662
6/6 [==============================] - 0s 2ms/step
>9, d1=0.736, d2=0.661
6/6 [==============================] - 0s 2ms/step
>10, d1=0.727, d2=0.671
6/6 [==============================] - 0s 2ms/step
>11, d1=0.723, d2=0.674
6/6 [==============================] - 0s 2ms/step
>12, d1=0.711, d2=0.680
6/6 [==============================] - 0s 3ms/step
>13, d1=0.702, d2=0.694
6/6 [=============================

In [62]:
noise = np.random.normal(0, 1, data.shape)
new_data = _df(data=trained_model.predict(noise))
original_data = scaler.inverse_transform(new_data)
print(type(original_data))
print(original_data.shape)

6/6 [==============================] - 0s 2ms/step
<class 'numpy.ndarray'>
(175, 5)


In [63]:
original_data_array = np.array(original_data)
columns = ["column_{}".format(i) for i in range(original_data_array.shape[1])]  # Replace with actual column names

# Convert NumPy array to pandas DataFrame
original_data_df = pd.DataFrame(original_data_array, columns=columns)

# Export DataFrame to CSV
original_data_df.to_csv('original_data.csv', index=False)